<a href="https://colab.research.google.com/github/archyyu/RNN-GPT/blob/main/studyRNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
#url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
url = "https://raw.githubusercontent.com/archyyu/publicResource/main/google.dev.en"
#url = "https://raw.githubusercontent.com/archyyu/publicResource/main/KDE4.en-es.en"
#url = "https://raw.githubusercontent.com/archyyu/publicResource/main/js"
response = requests.get(url)
data = response.text

In [ ]:
data

In [ ]:
chars = sorted(set(''.join(data)))
i2c = dict(enumerate(chars))
c2i = {c: i for i, c in i2c.items()}

def encode(s):
  il = []
  for c in s:
    il.append(c2i[c])
  return il

def decode(l):
  cs = []
  for i in l:
    cs.append(i2c[i])
  return ''.join(cs)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class RNN(nn.Module):

  def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
    super(RNN, self).__init__()
    self.emb = nn.Embedding(vocab_size, embedding_dim)
    self.i2h = nn.Linear(embedding_dim, hidden_size)
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.h2o = nn.Linear(hidden_size, output_size)
    self.bh = nn.Parameter(torch.zeros(hidden_size, 1,dtype=torch.float))
    self.bo = nn.Parameter(torch.zeros(output_size, 1,dtype=torch.float))

  def forward(self, input, hidden):
    embedding = self.emb(input)
    fist = self.i2h(embedding)
    second = self.h2h(hidden)
    print(fist.shape)
    print(second.shape)
    h = torch.tanh(fist + second + self.bh)
    y = self.h2o(h) + self.bo
    return y, h


In [ ]:
class DataLoader(object):

  def __init__(self, data, seq_length):
    self.data = data
    self.seq_length = seq_length
    self.chars = list(set(data))
    self.char_to_ix = {ch: i for i, ch in enumerate(self.chars)}
    self.ix_to_char = {i: ch for i, ch in enumerate(self.chars)}

  def __iter__(self):
    for i in range(len(self.data) - self.seq_length + 1):
      inputs = torch.tensor([self.char_to_ix[ch] for ch in self.data[i:i + self.seq_length]],dtype=torch.long)
      targets = torch.tensor([self.char_to_ix[ch] for ch in self.data[i + 1:i + self.seq_length + 1]],dtype=torch.float)
      yield inputs, targets

In [ ]:
seq_length = 25
data_loader = DataLoader(data, seq_length)

In [ ]:
for inputs, targets in data_loader:
  print("input:",inputs)
  print("output:",targets)
  break

input: tensor([55, 16, 17, 34,  9,  0, 54, 16,  9, 16, 23, 19, 49,  2, 41, 57, 19,  4,
        46, 17, 19,  0, 45, 19,  0])
output: tensor([16., 17., 34.,  9.,  0., 54., 16.,  9., 16., 23., 19., 49.,  2., 41.,
        57., 19.,  4., 46., 17., 19.,  0., 45., 19.,  0., 62.])


In [ ]:
chars = list(set(data))
len(chars)

65

In [ ]:
n_hidden = 128
embedding_dim = 10
chars = list(set(data))
rnn = RNN(len(chars), embedding_dim, n_hidden, len(chars))
learning_rate = 0.0005
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 2000

for epoch in range(n_epochs):
  h_prev = torch.zeros(n_hidden, n_hidden)
  for inputs, targets in data_loader:
    optimizer.zero_grad()
    outputs, h_prev = rnn(inputs, h_prev)
    loss = loss_fn(outputs, targets)
    loss.backward()
    optimizer.step()
  if epoch % 1000 == 0:
    print(loss)


torch.Size([25, 128])
torch.Size([128, 128])


RuntimeError: ignored

In [ ]:
import torch
import torch.nn as nn

# Define the vocabulary size and embedding dimension
vocab_size = 65  # Number of unique items (e.g., words)
embedding_dim = 128  # Dimensionality of the embedding space

# Create an instance of nn.Embedding
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Generate some example indices (batch size = 3, sequence length = 2)
indices = torch.tensor([17., 61., 34., 25., 28.,  6., 27., 61., 28., 61., 33., 41.,  1., 53.,
        50., 56., 41.,  7., 58., 34., 41.,  6., 47., 41.,  6.], dtype=torch.long)

# Use the embedding layer to get dense representations for the indices
embeddings = embedding_layer(indices)

# Print the result
print("Embeddings:")
print(embeddings)
embeddings.shape


Embeddings:
tensor([[-0.9908, -0.6424, -0.2864,  ...,  0.4558, -0.2979,  0.9670],
        [ 0.0865, -0.5986, -0.5198,  ..., -0.3778, -0.2373,  0.4017],
        [ 0.3628,  0.4025,  1.1648,  ..., -0.3780,  1.0158, -0.8076],
        ...,
        [-0.3223, -0.1076, -2.0110,  ...,  1.2723,  0.1564,  0.4164],
        [-0.6555, -0.4614, -1.4212,  ...,  0.0568, -0.5009,  0.4072],
        [-1.2549,  0.4909,  1.5384,  ...,  0.6061, -0.2331, -1.3316]],
       grad_fn=<EmbeddingBackward0>)


torch.Size([25, 128])